In [1]:
import pandas as pd # data processing
import numpy as np # working with arrays
import matplotlib.pyplot as plt # visualization
import seaborn as sns
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn import preprocessing
import math 

In [4]:
root = '/home/sihartist/Desktop/'
path = root + 'fraud-detection/dataset/clean_data.xlsx'

In [5]:
# Load the credit card transaction data
df = pd.read_excel(path,dtype = str)

In [11]:
def update_dtype():
    df['CLASS'] = df['CLASS'].astype(int)
    df['V10'] = df['V10'].astype(float)
    df['V9'] = df['V9'].astype(float)
    df['V24'] = df['V24'].astype(float)

#Object preprocessing 
def to_categorical(v_df):
    cols= v_df.select_dtypes(include=['object'])
    obj_cols=cols.columns.to_list()

    for col in obj_cols:
        check=len(v_df[col].unique())
        print(str(col)+" ",check)
        le = LabelEncoder()
        v_df[col] = le.fit_transform(v_df[col])
    return v_df

def process_date():
    #DATETIME
    list=['V6','V7','V23','V8']
    date_columns=df.filter(list)
    year_columns = pd.DataFrame()
    
    for l in list:
        date_columns[l] = pd.to_datetime(date_columns[l])
        date_columns[l+'_month'] = date_columns[l].dt.month    
        date_columns[l+'_day'] = date_columns[l].dt.day.astype(int)
        year_columns[l+'_year'] = date_columns[l].dt.year

    date_columns.drop(list, axis=1, inplace=True)
    df.drop(list, axis=1, inplace=True)

    #Normalize date cols
    min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-math.pi, math.pi - (2*math.pi)/31 ))
    date_columns_scaled = min_max_scaler.fit_transform(date_columns.values)
    date_columns_normalized = pd.DataFrame(date_columns_scaled, columns=date_columns.columns)
    date_columns_normalized = date_columns_normalized.applymap(lambda x: math.sin(x))

    new_df = pd.concat([df, date_columns_normalized, year_columns], axis=1)
    return new_df

In [12]:
update_dtype()
df_final=process_date()
df_final=to_categorical(df_final)

/home/sihartist/.local/lib/python3.8/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '26/12/2009' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/home/sihartist/.local/lib/python3.8/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '26/12/2010' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/home/sihartist/.local/lib/python3.8/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '22/06/2008' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/home/sihartist/.local/lib/python3.8/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '23/06/2008' in DD/MM/YYYY 

V1  7
V2  4
V3  101
V4  3
V12  1119
V13  4937
V16  124
V19  68888
V21  12
V22  6
V25  3
V26  999
V27  8


In [13]:
df_final.head()

,V1,V2,V3,V4,V9,V10,V12,V13,V16,V19,...,V7_month,V7_day,V23_month,V23_day,V8_month,V8_day,V6_year,V7_year,V23_year,V8_year
0,4,2,23,1,285328.0,33443.0,19,2976,44,20238,...,0.201299,0.980307,0.201299,0.980307,-0.525049,-0.937752,2009,2009,2009,2009
1,4,2,23,1,483993.0,652487.0,19,2976,44,1352,...,0.201299,0.980307,0.201299,0.980307,-0.368810,0.937752,2009,2009,2009,2009
2,4,2,23,1,713040.0,554381.0,19,2976,44,25276,...,0.201299,0.980307,0.201299,0.980307,-0.893709,-0.937752,2009,2009,2009,2009
3,4,2,23,1,592014.0,778437.0,19,2976,44,24482,...,0.201299,0.980307,0.201299,0.980307,-0.996172,-0.937752,2009,2009,2009,2009
4,4,2,23,1,18805.0,314278.0,19,2976,44,16915,...,0.201299,0.980307,0.201299,0.980307,-0.996172,-0.937752,2009,2009,2009,2009


In [15]:
df_final.to_excel(root + 'fraud-detection/dataset/preprocessing_data.xlsx', index=False)